# Annexe : Analyse de la base de données d'articles pour pouvoir récupérer les fréquences d'apparition des termes liés à l'inflation

## Avant-propos et Plan

Avant-propos : Ce notebook retrace les étapes à suivre pour accéder aux dictionnaires de fréquences d'utilisation des termes liés à l'inflation. Le parcours n'a pas été aussi direct, il a nécessité plusieurs essais infructueux, plusieurs fonctions pour tester différentes choses comme par exemple la structure de la base d'articles. Nous avons décidé de supprimer ces tentatives pour ne laisser que le code final par soucis de lisibilité.


Plan : Dans un premier temps, nous téléchargeons les données nécessaires à l'analyse du corpus. Ensuite, nous introduisons les fonctions principales pour cette partie. Pour continuer, on "teste" notre programme sur une petite partie du corpus pour voir comment sort le dictionnaire de fréquences. Pour finir, le code s'applique sur tout le corpus





NotaBene : La majeure partie des discussions et analyses sur les résultats sont faits dans le notebook principal. Vous pouvez trouver néanmoins des explications diverses sur le code.

NotaBene2 : Comme il est rappelé ci-dessous, il est conseillé d'appliquer les fonctions sur une année, voire une décennie au lieu de sur tout le corpus pour des questions de temps d'exécution. Vous pouvez trouver les temps d'exécution plus bas dans le notebook

## A/ Importation des données et fonctions nécessaires


### Ces fonctions se retrouve initialement dans le notebook 'gettingNewspapers.ipynb'. On les redéfinit ici pour que ce notebook annexe soit autonome.

In [1]:
import requests
import pandas as pd
import json
import os
import csv
import numpy as np
import tarfile
from dotenv import load_dotenv
from functools import partial


In [2]:
def load_credentials():
    load_dotenv()
    url="https://huggingface.co/api/datasets/dell-research-harvard/AmericanStories"
    token=os.getenv("HuggingFaceToken")
    headers = {
        "Authorization" : f"Bearer {token}"
    }
    return headers

### download_targz prend en argument les années que l'on veut télécharger et les télécharge dans le dossier *ArticlesTarGz*

In [3]:
def download_targz(years):
    if isinstance(years, str):
        years=[years]
    os.makedirs("ArticlesTarGz", exist_ok=True)
    for i in years:
        print(i, end=" ")
        file=f"faro_{i}.tar.gz"
        url=f"https://huggingface.co/datasets/dell-research-harvard/AmericanStories/resolve/main/{file}"
        wb=requests.get(url, headers=headers)
        print(wb.status_code)
        if wb.ok:
            with open("ArticlesTarGz/"+file, "wb") as f:
                f.write(wb.content)
        else:
            print(f"Error {wb.status_code} downloading {file}")

### extract_targz est une fonction qui permet d'extraire les articles des fichier en targz de la fonction précédente.

In [4]:
def extract_targz(years):
    if isinstance(years, str):
        years=[years]
    for i in years:
        print(i, end=" ")
        try :
            with tarfile.open(f"ArticlesTarGz/faro_{i}.tar.gz", "r:gz") as tar:
                tar.extractall(path=f"ArticlesTarGz/faro_{i}")
        except FileNotFoundError:
            print(f"Error extracting {i} The file is not found")

## B/ Fonctions principales pour récupérer les fréquences

Ici, on crée le dictionnaire avec les expressions que l'on considère comme relatives à l'inflation. Comme il est dit dans le notebook, nous nous sommes inspirés de l'étude faite par la Banque de France sur l'analyse de l'inflation perçue par la population.

Nous avons séparé les expressions par leur nombre de mots pour mieux s'y retrouver.

In [5]:
def creer_dictionnaire_inflation():
    #Cette fonction renvoie un dictionnaire de n-gram relatifs à l'inflation et aux prix

    dictionnaire = {
        "1-gram": [
            "inflation", "disinflation", "inflationary", "deflation", "prices", "cost", "wages", "currency",
            "money", "devaluation","recession", "stagflation", "economy", "market", "increase", "decrease", "cpi"
        ],
        "2-gram": [
            "price level", "wage growth", "economic downturn", "monetary policy",
            "cost increase", "cost reduction", "inflation expectations", "market prices", "inflation rate",
            "interest rates", "price stability", "consumption basket", "purchasing power"
        ],
        "3-gram": [
            "consumer price index", "rise in prices", "fall in prices",
            "cost of living", "money supply growth",
            "central bank policy", "economic price adjustments"
        ]
    }
    return dictionnaire

dictionnaire_inflation = creer_dictionnaire_inflation()


La fonction suivante a pour objectif de nettoyer les textes. Cela est particulièrement utile car, après analyse, nous nous sommes rendus compte que les articles contiennent de nombreux passages qui ne sont pas du texte (lien vers des images avec leurs positions dans le journal). Donc nous nettoyons le texte avant de vérifier si une expression apparaît dedans.

In [6]:
import re

def nettoyer_texte(text):
    #Renvoie le texte text nettoyé, c'est-à-dire sans majuscule et avec uniquement les mots du texte (en particulier, on enlève les parties entre accolades qui semblent ne pas être du texte)
    text = text.lower()
    text = re.sub(r'\{.*?\}', ' ', text)
    text = re.sub(r'[^a-zA-Z\s]', ' ', text) 
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

On teste notre fonction de nettoyage sur des exemples.

In [7]:
text_test_1 = "Miam le   price level        economic  price    adjustments     espace !!"
text_test_2 = "Texte, wouah inflation... { bla bla bizarre x1 : 258}     ."
text_test_3 = "normalent on ne change rien ici"
text_test_4 = " Edward II. and Leila Clemens, boy.Robert L. and Priscilla Craney, girl.Augustus Z. and Daisy Berry, girlJessie and Mary Anderson, boy.', 'byline': '', 'bbox_list': [{'x0': 2737, 'y0': 961, 'x1': 3267, 'y1': 1020}, {'x0': 2608, 'y0': 1038, 'x1': 3338, 'y1': 1433}], 'bbox': [2608, 961, 3338, 1433], 'full_article_id': 19, 'id': '19_1919-04-24_p22_sn84026749_00280764577_1919042401_0444.json'}, {'object_ids': [59, 19], 'headline': 'DOROTHY c. GILMORE.', 'article': 'With scores Of schoolmates andfriends present, funeral services arebeing held this afternoon for DorothyCaroline Gilmore, twelve years old.daughter Of Charles w. Gilmore,"

print(nettoyer_texte(text_test_1))
print(nettoyer_texte(text_test_2))
print(nettoyer_texte(text_test_3))
print(nettoyer_texte(text_test_4))

miam le price level economic price adjustments espace
texte wouah inflation
normalent on ne change rien ici
edward ii and leila clemens boy robert l and priscilla craney girl augustus z and daisy berry girljessie and mary anderson boy byline bbox list bbox full article id id p sn json object ids headline dorothy c gilmore article with scores of schoolmates andfriends present funeral services arebeing held this afternoon for dorothycaroline gilmore twelve years old daughter of charles w gilmore


### Maintenant, on code les fonctions qui permettent de sortir le dictionnaire des fréquences ainsi que la liste des textes contenant au moins une expression


On raisonne avec une fonction élémentaire. Une autre fonction se charge d'appliquer cette fonction élémentaire sur tous les articles que contient une brochure (un numéro d'un journal). On applique ensuite cette fonction sur tous les journaux de notre base de données.

On utilise un effet de bord pour former au fûr et à mesure notre dictionnaire de fréquences.

In [8]:
def detection_expression(article,expression):
    #Hyp : article est une chaîne de caractère, expression est un motif sous forme de chaîne de caractère
    #Renvoie True si l'article contient AU MINIMUM une fois expression, renvoie False sinon
    return article.lower().find(expression)!=-1

In [9]:
def filter_and_freq(dict_freq,detection_expression,title,date,articles,expression):
    #Cette fonction prend en entrée le dictionnaire de fréquences, la fonction ci-dessus, un numéro d'un journal et une expression
    #Elle renvoie le liste des articles du journal contenant au moins une fois la chaîne 'expression'

    #Effet de Bord : La fonction modifie le dictionnaire des fréquences et ces modifications se retrouvent directement dans le dictionnaire crée avant la fonction (voir ci-dessous)
    if not (date in dict_freq.keys()):
        dict_freq[date]={}
        dict_freq[date]["nbre_articles"] = 0
    
    if not(expression in dict_freq[date].keys()):
        dict_freq[date][expression] = 0

    for i in range(len(articles)): 
        n = dict_freq[date]["nbre_articles"]
        f = dict_freq[date][expression]

        if detection_expression(articles[i]['article'], expression):            
            dict_freq[date][expression]= f+1
        else :
            articles[i]=None
        dict_freq[date]["nbre_articles"] = n+1
    articles=[i for i in articles if i is not None]
    return (title,date,articles)

In [10]:
import os


def map_local(fonction,years,expression):
    #Cette fonction applique la fonction précédente à l'ensemble des journaux publiés dans la liste d'années years
    #Ici, l'expression est fixe.
    if isinstance(years, str):
        years=[years]
    acc=[]
    for i in years:
        print(i, end=" ")
        files=os.listdir(f"ArticlesTarGz/faro_{i}/mnt/122a7683-fa4b-45dd-9f13-b18cc4f4a187/ca_rule_based_fa_clean/faro_{i}")
        for j in files:
            try:
                with open(f"ArticlesTarGz/faro_{i}/mnt/122a7683-fa4b-45dd-9f13-b18cc4f4a187/ca_rule_based_fa_clean/faro_{i}/{j}") as f:
                    data = json.load(f)
                if data['full articles']!=[]:
                    try :
                        (title, date, articles) = fonction(data["lccn"]["title"],data['edition']['date'][:-3],data['full articles'],expression)

                        if articles != []:
                            acc.append((title,date,articles))

                    except:
                        (title, date, articles) = fonction("No titles found",data['edition']['date'][:-3],data['full articles'],expression)

                        if articles != []:
                            acc.append((title,date,articles))
            #Cette exception sert principalement à l'année 1919 qui possède un fichier json corrompu        
            except json.JSONDecodeError as e:
                print(f"Erreur lors du chargement du fichier : {e}")


    return acc
    

### Maintenant, on applique cette fonction ci-dessus pour chacune des expressions du dictionnaire d'expressions relatives à l'inflation 

In [11]:
def calculer_frequence_expression_inflation(years):
    #C'est la fonction principale de cette partie : Pour chacune des expressions du dictionnaire d'expression, elle applique la fonction ci-dessus pour obtenir l'ensemble des articles contenant une expression ainsi que le dictionnaire des fréquences
    compteur_nombre_expression = 0
    dictionnaire_articles_inflation = {}

    for categorie_n_gram in ["1-gram","2-gram","3-gram"]:  
        for expression in dictionnaire_inflation[categorie_n_gram]:
            print("\n\n On attaque l'expression : ", expression," . Années terminées : ")
            compteur_nombre_expression = compteur_nombre_expression + 1
            liste_articles = map_local(f_f,years,expression)

            dictionnaire_articles_inflation[expression] = liste_articles

    for date, sous_dictionnaire in frequences.items():
        if "nbre_articles" in sous_dictionnaire:
            sous_dictionnaire["nbre_articles"] = int(sous_dictionnaire["nbre_articles"]/compteur_nombre_expression)

    return dictionnaire_articles_inflation

## C/Application à une partie du corpus

Le temps d'exécution de ce code sur l'ensemble du corpus met plus de 3 jours à tourner (exactement, il met 5411 minutes). Pour pouvoir tout de même tester les fonctions de cette partie. On vous propose de le faire, soit sur une année, soit sur une décennie.

### Sur l'année 1929 :

(environ 7 minutes de traitement)

In [14]:
#On télécharge les fichiers nécessaires :

headers=load_credentials()
download_targz([1929])
print("Téléchargement terminé !")

extract_targz([1929])
print("\nExtraction terminée !")

#On crée le dictionnaire dans lequel on va mettre petit à petit les fréquences d'apparition de chaque expression
frequences = {}
f_f=partial(filter_and_freq, frequences, detection_expression)


dictionnaire_articles_inflation_1929 = calculer_frequence_expression_inflation([1929])

print("Analyse terminée !")

1929 200
Téléchargement terminé !
1929 

/tmp/ipykernel_64285/655446991.py:8: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=f"ArticlesTarGz/faro_{i}")



Extraction terminée !


 On attaque l'expression :  inflation  . Années terminées : 
1929 

 On attaque l'expression :  disinflation  . Années terminées : 
1929 

 On attaque l'expression :  inflationary  . Années terminées : 
1929 

 On attaque l'expression :  deflation  . Années terminées : 
1929 

 On attaque l'expression :  prices  . Années terminées : 
1929 

 On attaque l'expression :  cost  . Années terminées : 
1929 

 On attaque l'expression :  wages  . Années terminées : 
1929 

 On attaque l'expression :  currency  . Années terminées : 
1929 

 On attaque l'expression :  money  . Années terminées : 
1929 

 On attaque l'expression :  devaluation  . Années terminées : 
1929 

 On attaque l'expression :  recession  . Années terminées : 
1929 

 On attaque l'expression :  stagflation  . Années terminées : 
1929 

 On attaque l'expression :  economy  . Années terminées : 
1929 

 On attaque l'expression :  market  . Années terminées : 
1929 

 On attaque l'expression :  increas

On exporte les résultats : Le dictionnaire des fréquences ainsi que la liste des articles contenant les expressions

In [15]:
output_file = "frequences_expression_1929.csv"

#Ici, on exporte notre dictionnaire sous la forme d'un fichier csv pour pouvoir l'utiliser par la suite.
#On trie le dictionnaire par ordre chronolique pour une meilleure lisibilité
with open(output_file, mode="w", newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    headers = ["date"] + list(next(iter(frequences.values())).keys())
    writer.writerow(headers)
    sorted_items = sorted(frequences.items(), key=lambda x: x[0])  

    for date, sub_dict in sorted_items:
        row = [date] + [
            sub_dict.get(header, "") if not isinstance(sub_dict.get(header, ""), np.ndarray) 
            else sub_dict.get(header, "")[0] for header in headers[1:]
        ]
        writer.writerow(row)

In [17]:
output_file = "articles_inflation_1929.json"

#Ici, on exporte le dictionnaire avec, pour chaque expression, la liste des articles contenant l'expression. On l'exporte sous un fichier json par soucis de simplicité
with open(output_file, mode="w", encoding="utf-8") as file:
    json.dump(dictionnaire_articles_inflation_1929, file, ensure_ascii=False, indent=4)

In [18]:
print(frequences)

{'1929-11': {'nbre_articles': 46365, 'inflation': 0, 'disinflation': 0, 'inflationary': 0, 'deflation': 6, 'prices': 751, 'cost': 1538, 'wages': 209, 'currency': 41, 'money': 1623, 'devaluation': 0, 'recession': 23, 'stagflation': 0, 'economy': 72, 'market': 1407, 'increase': 1003, 'decrease': 154, 'cpi': 22, 'price level': 6, 'wage growth': 0, 'economic downturn': 0, 'monetary policy': 1, 'cost increase': 0, 'cost reduction': 0, 'inflation expectations': 0, 'market prices': 23, 'inflation rate': 0, 'interest rates': 24, 'price stability': 0, 'consumption basket': 0, 'purchasing power': 13, 'consumer price index': 0, 'rise in prices': 1, 'fall in prices': 0, 'cost of living': 8, 'money supply growth': 0, 'central bank policy': 0, 'economic price adjustments': 0}, '1929-08': {'nbre_articles': 45022, 'inflation': 0, 'disinflation': 0, 'inflationary': 0, 'deflation': 2, 'prices': 514, 'cost': 1517, 'wages': 151, 'currency': 51, 'money': 1486, 'devaluation': 0, 'recession': 15, 'stagflatio

On affiche le dictionnaire des fréquences sous la forme d'un dataframe.

In [19]:
frequences_data=pd.DataFrame(frequences).T
print(frequences_data)

         nbre_articles  inflation  disinflation  inflationary  deflation  \
1929-11          46365          0             0             0          6   
1929-08          45022          0             0             0          2   
1929-02          32626          0             0             0          3   
1929-09          35326          0             0             0          2   
1929-06          46991          0             0             0          8   
1929-01          34921          1             0             0          5   
1929-07          44949          0             0             0          3   
1929-10          40179          0             0             0          0   
1929-03          38402          0             0             0          5   
1929-04          45940          0             0             0          7   
1929-12          46530          0             0             0          5   
1929-05          42844          0             0             0          4   

         pr

## D/ Application sur le corpus intégral

### Attention : Cette partie s'exécute en environ 90h !

In [ ]:
#On télécharge les fichiers nécessaires :

headers=load_credentials()
download_targz([i for i in range(1919,1964)])
print("Téléchargement terminé !")

extract_targz([i for i in range(1919,1964)])
print("\nExtraction terminée !")

#On crée le dictionnaire dans lequel on va mettre petit à petit les fréquences d'apparition de chaque expression
frequences = {}
f_f=partial(filter_and_freq, frequences, detection_expression)


dictionnaire_articles_inflation = calculer_frequence_expression_inflation([i for i in range(1919,1964)])

print("Analyse terminée !")